# Bildklassifikation mit dem MURA-Datensatz

Bei der Klassifikation geht es darum, einem Bild ein Label aus einer vorgegebenen Menge an möglichen Labels zuzuweisen. Im Vergleich zur Segmentierung und Objekt Detection ist diese Aufgabe die einfachste.

In diesem Notebook wird ein Klassifikationsmodell für den MURA-Datensatz mit Keras und Tensorflow als Backend trainiert.

## Datensatz

Der [MURA](https://stanfordmlgroup.github.io/competitions/mura/) (**mu**sculoskeletal **ra**diographs) Datensatz besteht aus 40.561 Röntgenbildern von Oberkörperextremitäten der Kategorien: Ellbogen, Finger, Unterarm, Hand, Oberarm, Schulter, und Handgelenk. Zusätzlich ist jedem Bild das Label normal oder abnormal zugeordnet.

![XR_HAND](http://s1.saviola.de/screens/mura.png)

In diesem Jupyter-Notebook soll ein Model trainiert werden welches Bilder des MURA-Datensatzes in eine der sieben Extremitätsklassen einordnen kann.

## Jupyter-Notebook

Jupyter-Notebooks sind interaktive Python-Scripte, in denen Markdown und sogar Latex zur Dokumentation verwendet werden kann. In diesem Abschnitt sollen einerseits allgemeine Informationen wie Tastenkombinationen und übliche Workflows, andererseits aber auch für diesen Workshop und die verwendete Hardware spezifische Informationen im Umgang mit Python und Jupyter-Notebooks geliefert werden.

### Wichtiges auf einen Blick (a.k.a. TL;DR)

- **beim Wechseln auf ein anderes Notebook immer den Kernel beenden (Kernel -> Shutdown)**
- **Shift + Enter** zum Ausführen der aktiven Zelle
- **Strg + Shift + p** zum Öffnen der Kommandopalette
- **Shift + o** zum Togglen des Zell-Scrollings
- bei Fehlermeldungen im Zweifel **Kernel neustarten (Kernel -> Restart)**
- "Hilfe, ich sehe den Markdown-Code" -> **Shift + Enter** in der entsprechenden Zelle
- "Hilfe, ich bekommen ResourceExhaustion / OutOfMemory (OOM) Fehler" -> **Kernel neustarten, Kernel von anderen noch laufenden Notebooks herunterfahren** (oben links auf das Jupyter-Logo klicken, dann auf den Tab "Running")
- "Hilfe, mein Notebook ist kaputt" -> siehe **Notebook "Wiederherstellung"**
- "Passiert da noch was?" -> ist der **Kreis oben rechts neben "Python 3", ausgefüllt und dunkel** dann ist der Kernel noch am Arbeiten, ist er nicht gefüllt dann ist der Kernel untätig. Die aktuell laufende Zelle ist die von oben gesehen erste bei der auf der linken Seite "In[*]" anstelle von z.B. "In[5]" steht. Es kann aber passieren dass sich der Kernel aufhängt, in dem Fall einfach oben auf __Kernel -> Interrupt__ und die Zelle erneut ausführen


### Überblick & Workflow

Die einzigen beiden Shortcuts die man sich eigentlich nur merken muss sind

- **Shift + Enter** zum Ausführen einer Zelle, und
- **Strg + Shift + p** zum Öffnen der Kommandopalette, von der aus man dann direkt Zugriff auf alle möglichen Befehle hat, inklusive entsprechender Shortcuts

Ein weiterer nützlicher Shortcut ist **Shift + o**, welcher das **Scrolling für Zellenoutput** umschaltet.

Zum **Editieren einer Zelle** genügt ein Doppelklick in die Zelle, bei Code-Zellen reicht es zum Beenden des Editiermodus einfach außerhalb der Zelle zu klicken, bei Dokumentationszellen (wie dieser hier) ist eine Ausführung der Zelle nötig um die Code-Ansicht zu verlassen.

Während eine Zelle ausgeführt wird, wechselt der Kernel-Indikator oben rechts neben "Python 3" von einem hellen Kreis mit dunklem Rand zu einem ausgefüllten dunklen Kreis und springt wieder zurück sobald die Ausführung beendet ist. Wenn mehrere Zellen gleichzeitig ausgeführt wurden, kann man an dem Label in der linken Spalte ablesen, ob die Zelle fertig ausgeführt wurde (**In [ZAHL]:**) oder ob sie gerade ausgeführt wird bzw. auf Ausführung wartet (**In [*]:**). Zusätzlich wird nach Ausführung einer Zelle die Zellenausgabe unterhalb der Zelle angezeigt.

Um den Überblick zu behalten kann es manchmal sinnvoll sein, die **Zellenausgabe zu löschen**. Dies kann u.a. auf diesen beiden Wegen erfolgen:

- oben auf Cell -> Current Outputs / All outputs -> Clear
- oben auf Kernel -> Restart & Clear Output

Der Kernel ist für die Ausführung des Python-Codes zuständig und behält den Kontext (also belegte Variablen, definierte Funktionen und belegter Speicher) seit dem letzten Kernel-(Neu)start. Dies kann zu Problemen führen wenn Zellen in anderer Reihenfolge ausgeführt werden oder Zellen übersprungen werden in denen Variablen oder Funktionen definiert werden die im weiteren Verlauf des Scripts benötigt werden, aber auch wenn **ein anderes Jupyter-Notebook gestartet wird**, da dafür ein weiterer Kernel gestartet wird.

Deswegen beim **Wechseln auf ein anderes Notebook** immer den **Kernel herunterfahren oder neustarten** (oben Kernel -> Restart/Shutdown).

# 1. Imports und Tensorflow-Setup

Neben den benötigten Imports wird wird hier Tensorflow angewiesen, nur maximal 40% des GPU-RAMs zu belegen, damit mehrere Trainingsprozesse gleichzeitig auf jeder GPU ausgeführt werden können. Überspringt man diesen Schritt, dann reserviert Tensorflow den gesamten Grafikspeicher. Ein wichtiger Faktor für den beim Training benötigen Grafikspeicher ist die gewählte Batch-Größe, auf die später im Script noch genauer eingangen wird.

Bitte diese Zahl nicht ändern, da mehrere Trainingsprozesse auf einer GPU ausgeführt werden und es zu Crashes kommen kann wenn der GPU-Speicher voll läuft.

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image as Image
import matplotlib.pyplot as plt
import logging
from scipy.interpolate import make_interp_spline, BSpline

# Make sure TF does not print confusing warnings
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UserWarning)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
logging.getLogger("tensorflow").setLevel(logging.ERROR)
import tensorflow as tf
from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False
tf.logging.set_verbosity(tf.logging.INFO)

from tensorflow import ConfigProto
# Tell TF to not use all GPU RAM
config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = .40
session = tf.Session(config=config)
print("Tensorflow setup done")

from tensorflow.python.keras import models
from tensorflow.python.keras import layers
from tensorflow.python.keras.callbacks import Callback
from tensorflow.python.keras.callbacks import ModelCheckpoint
from tensorflow.python.keras.applications.resnet50 import ResNet50

from tensorflow.python.keras import backend as K

from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

# Functions for visualization
# See https://www.tensorflow.org/tutorials/images/hub_with_keras
def image_to_network_input(filename):
    im = Image.open(filename)
    plt.imshow(np.asarray(im))

    # Convert to RGB and resize
    im_converted = im.convert(mode='RGB').resize((IMAGE_SIZE,IMAGE_SIZE))

    # Convert to array and add batch dimension
    X_test = np.array(im_converted)/255.0
    X_test = X_test[np.newaxis, ...]
    
    return X_test

def print_prediction_result(result):
    print('Predictions:')
    prediction_sum = np.sum(result)
    i = 0
    for x in np.nditer(result):
        print('{}: {:.3f}%'.format(CLASSES[i], x/prediction_sum*100))
        i+=1
        
    predicted_class = CLASSES[np.argmax(result[0], axis=-1)]
    print('Predicted class: ' + predicted_class)

def smooth(values, num_points = 100):
    xnew = np.linspace(0,len(values)-1,num_points) #100 represents number of points to make between T.min and T.max
    spl = make_interp_spline(list(range(len(values))), values, k=3) #BSpline object
    values_smooth = spl(xnew)
    
    return xnew, values_smooth
    
class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.loss = []
        self.acc = []

    def on_batch_end(self, batch, logs={}):
        self.loss.append(logs.get('loss'))
        self.acc.append(logs.get('acc'))


def random_crop(img, random_crop_size):
    # Note: image_data_format is 'channel_last'
    assert img.shape[2] == 3
    height, width = img.shape[0], img.shape[1]
    dy, dx = random_crop_size
    x = np.random.randint(0, width - dx + 1)
    y = np.random.randint(0, height - dy + 1)
    return img[y:(y+dy), x:(x+dx), :]


def crop_generator(batches, crop_length):
    """Take as input a Keras ImageGen (Iterator) and generate random
    crops from the image batches generated by the original iterator.
    """
    while True:
        batch_x, batch_y = next(batches)
        batch_crops = np.zeros((batch_x.shape[0], crop_length, crop_length, 3))
        for i in range(batch_x.shape[0]):
            batch_crops[i] = random_crop(batch_x[i], (crop_length, crop_length))
        yield (batch_crops, batch_y)

def find_last(model_directory):
        """Finds the last checkpoint file of the last trained model in the
        model directory.
        Returns:
            The path of the last checkpoint file
        """
        
        checkpoints = next(os.walk(model_directory))[2]
        checkpoints = filter(lambda f: f.endswith(".hdf5"), checkpoints)
        checkpoints = sorted(checkpoints)
        if not checkpoints:
            import errno
            raise FileNotFoundError(
                errno.ENOENT, "Could not find weight files in {}".format(model_directory))
        checkpoint = os.path.join(model_directory, checkpoints[-1])
        return checkpoint
        
        
print("Imports and setup done")

ROOT_DIR = os.path.abspath("/workspace")
DATA_DIR = os.path.abspath("/data")
MODEL_DIR = os.path.join(ROOT_DIR, "models/classification")
PRETRAINED_MODEL_DIR = os.path.abspath("/models")

# Solutions
#NUM_CLASSES =
#LEARN_RATE = 
#CONV_BASE_TRAINABLE = 

print("Directories:")
print("Root directory:", ROOT_DIR)
print("Model directory:", MODEL_DIR)
print("Pre-trained model directory:", PRETRAINED_MODEL_DIR)
print("Datasets directory:", DATA_DIR)

Die wichtigsten Ordner sind:

- `ROOT_DIR`: Das Hauptverzeichnis in dem sich die Jupyter-Notebooks befinden
- `MODEL_DIR`: Verzeichnis in dem trainierte Modelle abgespeichert werden
- `PRETRAINED_MODEL_DIR`: Verzeichnis für vortrainierte Modelle (read-only)
- `DATA_DIR`: Verzeichnis in dem sich die Datensätze befinden

Hier kann mit einfachen Linux-Kommandozeilenbefehlen die Ordnerstruktur untersucht werden

In [ ]:
!ls -l /data
!ls -l /data/MURA-v1.1
!ls -l /data/MURA-v1.1/train
!ls -l /workspace

# 2. Datensatz

Hier wird der Datensatz definiert. Es gibt in Keras und Tensorflow viele verschiedene Wege Daten einzulesen, in diesem Beispiel wird der [ImageDataGenerator](https://keras.io/preprocessing/image/) verwendet, der aufgrund eines Verzeichnis und Vorverarbeitungsoptionen Bilder in der folgenden Ordnerstruktur `/path/to/dataset/split/class/image.png` einliest und vorverarbeitet (Data Augmentation). Die verschiedenen Optionen zur Data Augmentation können unter dem o.g. Link nachgelesen werden und können später zur Optimierung genutzt werden.

In unserem Beispiel ist der Name des Datensatz-Ordners `MURA-v1.1` und die möglichen __Splits__ sind `train` und `valid`, die Klassen entsprechen den Ordnernamen innerhalb der Unterverzeichnisse `train/` und `valid/`.

Eine wichtige Variable hier ist `BATCH_SIZE`, welche bestimmt wie viele Bilder gleichzeitig in den Grafikspeicher geladen werden müssen. Wählt man diesen Wert zu hoch, so wird der reservierte Grafikspeicher nicht ausreichen, wählt man ihn zu niedrig so wird das Training sehr langsam. Für unser Beispiel soll dieser Wert auf 16 gesetzt werden.

In [ ]:
##
# Create dataset
##

print("Start preparing dataset... please wait, this may take a while")

BATCH_SIZE = 16 #8
IMAGE_SIZE = 128 #400
PRE_CROPPED_SIZE = 150 #450

# Dataset directory
DATASET_DIR = '/data/MURA-v1.1/'

CLASSES = ['XR_ELBOW', 'XR_FINGER', 'XR_FOREARM', 'XR_HAND', 'XR_HUMERUS', 'XR_SHOULDER', 'XR_WRIST']

# make sure these numbers are correct, you can count images using something like
# find path/to/dataset/split -name ".png" | wc -l
# or you just run the script once and look for the
# "Found X images belonging to Y classes." lines
NUM_TRAINING_SAMPLES = 36808
NUM_VALIDATION_SAMPLES = 3197

# Data augmentation
# see https://keras.io/preprocessing/image/
# see https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html
train_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        #interpolation_order=2,
        fill_mode='nearest')

# only rescale validation data
validation_datagen = ImageDataGenerator(rescale=1. / 255)

# this is a generator that will read pictures found in subfolers of '/train',
# in the dataset directory and indefinitely generate batches of augmented image
# data
print("Training data:")
train_generator_raw = train_datagen.flow_from_directory(
        DATASET_DIR + '/train',  # this is the target directory
        target_size=(PRE_CROPPED_SIZE, PRE_CROPPED_SIZE),  # all images will be resized to 128x128
        batch_size=BATCH_SIZE,
        interpolation="lanczos",
        class_mode='categorical')

train_generator = crop_generator(train_generator_raw, IMAGE_SIZE)

# this is a similar generator, for validation data
print("Validation data:")
validation_generator = validation_datagen.flow_from_directory(
        DATASET_DIR + '/valid',
        target_size=(IMAGE_SIZE, IMAGE_SIZE),
        batch_size=BATCH_SIZE,
        interpolation="lanczos",
        class_mode='categorical')

print("Dataset prepared")

## 2.1. Bilder anzeigen lassen

Im Folgedenen werden vier zufällige Bilder aus dem (Trainings-)Datensatz in Originalauflösung extrahiert. Um stattdessen Bilder aus dem Validierungsdatensatz anzuzeigen, einfach das `'/train'` in der ersten Zeile zu `'/valid'` ändern.

Dieser Block kann beliebig oft ausgeführt werden, die ausgewählten Bilder werden jedes Mal zufällig neu ausgewählt.

In [ ]:
example_generator = ImageDataGenerator().flow_from_directory(DATASET_DIR + '/train', batch_size=1, class_mode='categorical')

NUM_IMAGES = 4
i = 1
for x_batch, y_batch in example_generator:
    plt.figure()
    plt.title(CLASSES[np.where(y_batch[0] == 1.)[0][0]])
    plt.imshow(x_batch[0].astype(int))
    i += 1
    
    if i > NUM_IMAGES:
        break

<a id='from_scratch'></a>

# 3. Modell-Erstellung (from scratch)

Falls Sie das Training from Scratch bereits durchgeführt haben, klicken Sie [hier](#pre_trained) um zum nächsten Schritt zu springen.

Als nächstes wird das Modell erstellt. In der Variable `MODEL_DIR` ist das Verzeichnis angegeben, in dem das trainierte Modell gespeichert und (beim Fortsetzen des Trainings) geladen werden soll.

`conv_base` enthält das Basis-Modell, in diesem Fall ein [ResNet50](https://www.kaggle.com/keras/resnet50), andere Möglichkeiten sind auf der [keras.io](https://keras.io/applications/)-Webseite aufgelistet (dafür müssen aber auch oben die Imports angepasst werden).

Auf das Basis-Modell wird eine neue fully-connected Schicht mit 1000 Knoten gesetzt, und darüber eine weitere fully-connected Ausgabeschicht deren Anzahl Knoten der Anzahl Klassen entspricht.

Abschließend wird das Modell kompiliert, wobei die Loss-Funktion, ein Optimizer (hier kann die Lernrate angepasst werden, s.u.), und die zu generierenden Metriken übergeben werden.

Hier wird direkt das komplette Modell trainiert, da alle Schichten zufällig initialisiert wurden und ein zweiphasiges Training daher nicht sinnvoll ist.

### Optimizer und Lernrate

Keras unterstützt [eine Reihe](https://keras.io/optimizers/) von Optimizern mit verschiedenen Parametern. Optimizer sind für die Anpassung der Gewichte zuständig. Wir werden für unser Training zwei verschiedene Optimizer nutzen:

- [Adam](https://arxiv.org/abs/1412.6980v8)
- [Stochastic Gradient Descent (SGD)](https://en.wikipedia.org/wiki/Stochastic_gradient_descent)

Die wichtigsten Parameter die von beiden Optimizern unterstützt werden:

- `lr`, die Lernrate
- `decay`, learning rate decay

Für die Lernrate liegen übliche zwischen `0.001` und `0.00001`. Größere Lernrate ermöglichen schnelleres Training, können aber auch zu problematischen Sprüngen im Lösungsraum und Divergenz (Fehler gleichbleibend schlecht) führen. Kleinere Lernraten eignen sich für das Fine-Tuning.

Der learning rate decay sorgt dafür, dass die Lernrate im Laufe des Trainings immer geringer wird. Die Idee ist mit einer großen Lernrate zu starten um so schnell in die Nähe eines lokalen Minimums zu gelangen, und dann mit einer geringeren Lernrate diesem lokalen Minimum möglichst nahe zu kommen. Der Wert sollte hier abhängig von der Anzahl Epochen und Batches gewählt werden. Die Formel für die Berechnung der aktuellen Lernrate ist

\begin{equation*}
l = l_{init} * \frac{1}{1 + d * i}
\end{equation*}

mit $l$ = Lernrate, $l_{init}$ = initiale Lernrate, $d$ = decay, $i$ = Iterationen (Anzahl verarbeiteter Batches).

Hier ein paar Beispiele mit verschiedenen Lernraten und $d = 0.00025$:

<style type="text/css" rel="stylesheet">
.foo table { width: 300px; }
</style>
<div class="foo">
    
| Epoche | &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; Formel &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; | $l$ bei $l_{init} = 0.001$ | $l$ bei $l_{init} = 0.0005$ |
| ---------- | ---------- | ---------- | ---------- |
| 0 | $l_{init} * \frac{1}{1 + 0.00025 * 0*2300}$ | &nbsp; &nbsp; 0.001 &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; | &nbsp; &nbsp; &nbsp; 0.0005 &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; |
| 1 | $l_{init} * \frac{1}{1 + 0.00025 * 1*2300}$ | &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; 0.000634921 | &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; 0.00031746 &nbsp; |
| 2 | $l_{init} * \frac{1}{1 + 0.00025 * 2*2300}$ | &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; 0.000465116 | &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;  0.000232558 |
| 3 | $l_{init} * \frac{1}{1 + 0.00025 * 3*2300}$ | &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; 0.000366972 | &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;  0.000183486 |
| 4 | $l_{init} * \frac{1}{1 + 0.00025 * 4*2300}$ | &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; 0.000303030 | &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;  0.000151515 |
| 5 | $l_{init} * \frac{1}{1 + 0.00025 * 5*2300}$ | &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; 0.000258065 | &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;  0.000129032 |

</div>

Aufgaben:

- Ersetzen Sie die Variable `NUM_CLASSES` durch die Anzahl Klassen
- Ersetzen Sie die Variable `LEARN_RATE` durch eine geeignete Lernrate

In [ ]:
##
# Create model
##

print("Creating model")

os.makedirs(MODEL_DIR, 0o777, exist_ok=True)
print("model_dir: ", os.path.realpath(MODEL_DIR))

model_file = os.path.join(MODEL_DIR, "model_from_scratch.h5")

# Create Keras model
# see https://keras.io/applications/
conv_base = ResNet50(weights=None,
                  include_top=False,
                  input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3))

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
# Add more layers here if needed
model.add(layers.Dense(1000, activation='relu'))
# The number of nodes in the last layer should be equal to the number of classes
model.add(layers.Dense(NUM_CLASSES, activation='softmax'))
# Enable training for the base model
conv_base.trainable = True

# Here you can adjust the learning rate
# see https://keras.io/optimizers/
optimizer = tf.keras.optimizers.Adam(lr=LEARN_RATE, decay=0.0025)

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

print("Model successfully created")


## 3.1 Training (from scratch)

Das eigentliche Training wird über die Funktion `fit_generator` gestartet, welchem die zuvor erstellten Generatoren übergeben werden.

Zusätzlich muss die Anzahl Epochen (in diesem Fall eine Epoche) sowie die Anzahl Schritte pro Epoche (weil aus einem Generator nicht die Anzahl der Trainings- und Validierungsbilder extrahiert werden kann) angegeben werden. Diese entspricht der Anzahl Trainingsbilder geteilt durch die Batch-Größe, in diesem Fall also `36800 / 16 = 2300`.

Dies kann einige Zeit dauern. Ob der Kernel im Hintergrund noch aktiv ist lässt sich in dem Kreis oben rechts erkennen – solange der Kreis ausgefüllt ist, läuft das Script noch.

In [ ]:
print("Start training from scratch")
history = LossHistory()
filepath = os.path.join(MODEL_DIR, "from_scratch_{epoch:02d}-{val_loss:.2f}.hdf5")
model_checkpoint = ModelCheckpoint(filepath)
model.fit_generator(
        generator=train_generator,
        steps_per_epoch=NUM_TRAINING_SAMPLES // BATCH_SIZE,
        epochs=1,
        validation_data=validation_generator,
        validation_steps=NUM_VALIDATION_SAMPLES // BATCH_SIZE,
        callbacks=[history])
models.save_model(model, filepath = model_file)
#evaluation = model.evaluate_generator(train_generator, NUM_TRAINING_SAMPLES // BATCH_SIZE)
#print("Training loss:", evaluation[0], "\nTraining accuracy: {:.2f}%".format(evaluation[1]*100))
#evaluation = model.evaluate_generator(validation_generator, NUM_VALIDATION_SAMPLES // BATCH_SIZE)
#print("Validation loss:", evaluation[0], "\nValidation accuracy: {:.2f}%".format(evaluation[1]*100))
print("Finished training, model saved to: " + MODEL_DIR)

Die Ausgabe des Trainingsscripts enthält folgende Informationen:

- `8/2300`: Das aktuelle Minibatch. Wie oben beschrieben ergibt sich die Zahl 30 aus der Anzahl Trainingsbilder durch die Batch-Größe: `36800 / 16 = 2300`
- `ETA: 1:55`: Erwartete Restzeit
- `413s 180ms/step`: Gesamtdauer für die Trainingsepoche sowie durchschnittliche Dauer eines Minibatches
- `loss: 1.3854`: Wert der zu minimierenden Verlustfunction (in diesem Fall [Cross Entropy](https://en.wikipedia.org/wiki/Cross_entropy)) auf dem letzten Minibatch
- `acc: 0.4926`: Accuracy auf dem aktuellen Trainings-Minibatch, also prozentualer Anteil korrekt klassifizierter Bilder
- `val_loss: 1.5387`: Wert der Verlustfunction auf den gesamten Validierungsdaten, wird nur einmal pro Epoche generiert
- `val_acc: 0.5148`: Accuracy auf den gesamten Validierungsdaten, wird nur einmal pro Epoche generiert

Wichtig ist hier ein Vergleich zwischen den Werten für die Trainings- und Validierungsdaten. Wenn diese Werte weit auseinander gehen dann liegt Overfitting vor, die Trainingsdaten werden also auswendig gelernt.

<a id='visualization'></a>
## 3.2 Visualisierung (from scratch)

Nach dem Training kann mit der `LossHistory`-Instanz eine Visualisierung der Klassifikationsergebnisse stattfinden.

Es werden hier nur die Loss und Accuracy der Trainingsbatches angezeigt, da der Validierungsfehler nur einmal am Ende jeder Epoche (in diesem Fall also insgesmat nur ein mal) berechnet wird.

In [ ]:
# summarize history for accuracy
plt.plot(*smooth(history.acc, 100))
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('step')
plt.legend(['train'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(*smooth(history.loss, 100))
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('step')
plt.legend(['train'], loc='upper left')
plt.show()

### 3.2.1 Einzelinferenz (Trainingsbilder, from scratch)

Spannender hingegen ist die anschließende Einzelinferenz: was genau macht unser Netz aus bestimmten Bildern der Trainingsmenge?

Es werden hier vier (Variable `IMAGES_TO_SHOW`) zufällige Bilder aus der Trainingsmenge genommen.

Über die Variable `NUM_WRONG` kann festgelegt werden, wie viele falsch klassifizierte Bilder mindestens ausgewählt werden sollen.

In [ ]:
example_generator = ImageDataGenerator().flow_from_directory(DATASET_DIR + '/train', batch_size=1, target_size=(IMAGE_SIZE,IMAGE_SIZE), class_mode='categorical')

IMAGES_TO_SHOW = 4
NUM_WRONG = 2
i = 0
j = 0
for x_batch, y_batch in example_generator:
    x_input = x_batch[0]/255.
    x_input = x_input[np.newaxis, ...]
    result = model.predict(x_input)
    
    predicted_class = CLASSES[np.argmax(result[0], axis=-1)]
    correct_class = CLASSES[np.where(y_batch[0] == 1.)[0][0]]
    
    if j >= IMAGES_TO_SHOW - NUM_WRONG and NUM_WRONG > 0 and predicted_class == correct_class:
        continue
        
    i += 1
    
    if predicted_class == correct_class:
        j += 1
    
    print("\n\nImage", i)
    plt.figure()
    plt.title(CLASSES[np.where(y_batch[0] == 1.)[0][0]])
    plt.imshow(x_batch[0].astype(int))
    
    
    print_prediction_result(result)
    print('Correct class:', CLASSES[np.where(y_batch[0] == 1.)[0][0]])
    plt.show()
    
    if i >= IMAGES_TO_SHOW:
        break



Hier wird das gleiche für Bilder aus dem Validierungsdatensatz durchgeführt.

In [ ]:
example_generator = ImageDataGenerator().flow_from_directory(DATASET_DIR + '/valid', batch_size=1, target_size=(IMAGE_SIZE,IMAGE_SIZE), class_mode='categorical')

IMAGES_TO_SHOW = 4
NUM_WRONG = 0
i = 0
j = 0
for x_batch, y_batch in example_generator:
    x_input = x_batch[0]/255.
    x_input = x_input[np.newaxis, ...]
    result = model.predict(x_input)
    
    predicted_class = CLASSES[np.argmax(result[0], axis=-1)]
    correct_class = CLASSES[np.where(y_batch[0] == 1.)[0][0]]
    
    if j >= IMAGES_TO_SHOW - NUM_WRONG:
        continue
    
    if predicted_class == correct_class:
        j += 1
    
    print("\n\nImage", i+1)
    plt.figure()
    plt.title(CLASSES[np.where(y_batch[0] == 1.)[0][0]])
    plt.imshow(x_batch[0].astype(int))
    
    
    print_prediction_result(result)
    print('Correct class:', CLASSES[np.where(y_batch[0] == 1.)[0][0]])
    plt.show()
    
    i += 1
    
    if i > IMAGES_TO_SHOW:
        break



<a id='pre_trained'></a>
# 4. Modell-Erstellung (pre-trained)

Nachdem wir uns das Training from scratch angesehen haben soll nun ein Training basierend auf einem vortrainierten Modell stattfinden. Die in Keras mitgelieferten Modelle sind mit dem ImageNet-Datensatz vortrainiert. Um ein vortrainiertes Netz zu nutzen muss bei der Erstellung der `conv_base` der `weights` Parameter auf `imagenet` gesetzt werden.

Wichtig ist hierbei `conv_base.trainable` zunächst auf `False` zu setzen, wodurch nur die neu hinzugefügten Schichten für eine Epoche trainierten werden.

Falls der Validierungsfehler nach dem ersten Trainingsschritt sehr schlecht ist, dann ist das kein Grund zur Sorge – dieser wird spätestens nach dem zweiten Trainingsschritt dem Trainingsfehler sehr ähnlich sein.

Falls ein `ResourceExhaustion` bzw. `OOM`-Error auftritt, muss der Kernel neustartet werden `"restart the kernel (with dialog)"`. Anschließend alle Zellen einschließlich der Datensatzerstellung (aber nicht das Training from scratch) ausführen und dann hier fortfahren. Klicken Sie dazu [hier](#from_scratch), dann oben auf `Cell -> Run all above`, und anschließend auf den Link zum Überspringen des Schrittes, um wieder hierhin zu gelangen.

In [ ]:
print('Creating pre-trained model, this may take a while...')

# Clear session memory to avoid OOM
K.clear_session()

# Create Keras model
# see https://keras.io/applications/
conv_base = ResNet50(weights='imagenet',
                  include_top=False,
                  input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3))

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
# Add more layers here if needed
model.add(layers.Dense(1000, activation='relu'))
# The number of nodes in the last layer should be equal to the number of classes
model.add(layers.Dense(NUM_CLASSES, activation='softmax'))

print("Model created")

## 4.1 Training neue Schichten (pre-trained)


Aufgaben:

- Frieren Sie die Gewichte der vortrainierten Schichten über den Parameter `conv_base.trainable` ein, indem Sie die Variable `CONV_BASE_TRAINABLE` durch `True` oder `False` ersetzen

In [ ]:
# Disable training for the pre-trained model
conv_base.trainable = CONV_BASE_TRAINABLE

# Here you can adjust the learning rate
# see https://keras.io/optimizers/
optimizer = tf.keras.optimizers.Adam(lr=LEARN_RATE, decay=0.00025, clipnorm=1.)

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

print('Training new model')

history = LossHistory()
filepath = os.path.join(MODEL_DIR, "pre_trained_heads_{epoch:02d}-{val_loss:.2f}.hdf5")
model_checkpoint = ModelCheckpoint(filepath)

print("Start training: only new layers")
# First train only top layers
model.fit_generator(
        train_generator,
        steps_per_epoch=NUM_TRAINING_SAMPLES // BATCH_SIZE,
        epochs=1,
        callbacks=[history, model_checkpoint],
        validation_data=validation_generator,
        validation_steps=NUM_VALIDATION_SAMPLES // BATCH_SIZE)

#models.save_model(model, filepath = model_file)
#evaluation = model.evaluate_generator(train_generator, NUM_TRAINING_SAMPLES // BATCH_SIZE)
#print("Training loss:", evaluation[0], "\nTraining accuracy: {:.2f}%".format(evaluation[1]*100))
#evaluation = model.evaluate_generator(validation_generator, NUM_VALIDATION_SAMPLES // BATCH_SIZE)
#print("Validation loss:", evaluation[0], "\nValidation accuracy: {:.2f}%".format(evaluation[1]*100))
print("Finished training, model saved to: " + MODEL_DIR)

<a id='visualization'></a>
## 4.2 Visualisierung nach Training neuer Schichten (pre-trained)

Nach dem Training kann mit der `LossHistory`-Instanz eine Visualisierung der Klassifikationsergebnisse stattfinden.

Diese Visualisierung zeigt jederzeit den Verlauf des zuletzt durchgeführten Trainings an, Sie können also nach jedem Training hierhin zurück kehren und diesen Code-Block erneut ausführen.

In [ ]:
# summarize history for accuracy
plt.plot(*smooth(history.acc, 100))
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('step')
plt.legend(['train'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(*smooth(history.loss, 100))
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('step')
plt.legend(['train'], loc='upper left')
plt.show()

### 4.2.1 Einzelinferenz nach Training neuer Schichten (pre-trained)

Spannender hingegen ist die anschließende Einzelinferenz: was genau macht unser Netz aus bestimmten Bildern der Trainingsmenge?

Es werden hier vier (Variable `IMAGES_TO_SHOW`) zufällige Bilder aus der Trainingsmenge genommen.

Über die Variable `NUM_WRONG` kann festgelegt werden, wie viele falsch klassifizierte Bilder mindestens ausgewählt werden sollen.

In [ ]:
example_generator = ImageDataGenerator().flow_from_directory(DATASET_DIR + '/train', batch_size=1, target_size=(IMAGE_SIZE,IMAGE_SIZE), class_mode='categorical')

IMAGES_TO_SHOW = 4
NUM_WRONG = 0
i = 0
j = 0
for x_batch, y_batch in example_generator:
    x_input = x_batch[0]/255.
    x_input = x_input[np.newaxis, ...]
    result = model.predict(x_input)
    
    predicted_class = CLASSES[np.argmax(result[0], axis=-1)]
    correct_class = CLASSES[np.where(y_batch[0] == 1.)[0][0]]
    
    if j >= IMAGES_TO_SHOW - NUM_WRONG and NUM_WRONG > 0 and predicted_class == correct_class:
        continue
    
    if predicted_class == correct_class:
        j += 1
    
    i += 1
    
    print("\n\nImage", i)
    plt.figure()
    plt.title(CLASSES[np.where(y_batch[0] == 1.)[0][0]])
    plt.imshow(x_batch[0].astype(int))
    
    
    print_prediction_result(result)
    print('Correct class:', CLASSES[np.where(y_batch[0] == 1.)[0][0]])
    plt.show()
    
    if i >= IMAGES_TO_SHOW:
        break



Hier wird das gleiche für Bilder aus dem Validierungsdatensatz durchgeführt.

In [ ]:
example_generator = ImageDataGenerator().flow_from_directory(DATASET_DIR + '/valid', batch_size=1, target_size=(IMAGE_SIZE,IMAGE_SIZE), class_mode='categorical')

IMAGES_TO_SHOW = 4
NUM_WRONG = 0
i = 0
j = 0
for x_batch, y_batch in example_generator:
    x_input = x_batch[0]/255.
    x_input = x_input[np.newaxis, ...]
    result = model.predict(x_input)
    
    predicted_class = CLASSES[np.argmax(result[0], axis=-1)]
    correct_class = CLASSES[np.where(y_batch[0] == 1.)[0][0]]
    
    if j >= IMAGES_TO_SHOW - NUM_WRONG:
        continue
    
    if predicted_class == correct_class:
        j += 1
    
    print("\n\nImage", i+1)
    plt.figure()
    plt.title(CLASSES[np.where(y_batch[0] == 1.)[0][0]])
    plt.imshow(x_batch[0].astype(int))
    
    
    print_prediction_result(result)
    print('Correct class:', CLASSES[np.where(y_batch[0] == 1.)[0][0]])
    plt.show()
    
    i += 1
    
    if i > IMAGES_TO_SHOW:
        break



## 4.3 Fine-Tuning (pre-trained)


Im zweiten Schritt wird nun das gesamte Modell für eine Epoche trainiert (`conv_base.trainable` wird auf `True` gesetzt). Als Optimizer wird nun SGD verwendet.

In [ ]:
conv_base.trainable = True

try:
    model.load_weights(find_last(MODEL_DIR))
    print("Loading weights from {}".format(MODEL_DIR))
except FileNotFoundError:
    pass

optimizer = tf.keras.optimizers.SGD(lr=LEARN_RATE, decay=0.000025)

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

try: history
except NameError: history = LossHistory()

filepath = os.path.join(MODEL_DIR, "pre_trained_full_{epoch:02d}-{val_loss:.2f}.hdf5")
model_checkpoint = ModelCheckpoint(filepath)

print("Start training from pre-trained: fine-tuning all layers")
model.fit_generator(
        generator=train_generator,
        steps_per_epoch=NUM_TRAINING_SAMPLES // BATCH_SIZE,
        epochs=1,
        validation_data=validation_generator,
        validation_steps=NUM_VALIDATION_SAMPLES // BATCH_SIZE,
        callbacks=[history, model_checkpoint])
#evaluation = model.evaluate_generator(train_generator, NUM_TRAINING_SAMPLES // BATCH_SIZE)
#print("Training loss:", evaluation[0], "\nTraining accuracy: {:.2f}%".format(evaluation[1]*100))
#evaluation = model.evaluate_generator(validation_generator, NUM_VALIDATION_SAMPLES // BATCH_SIZE)
#print("Validation loss:", evaluation[0], "\nValidation accuracy: {:.2f}%".format(evaluation[1]*100))
print("Finished training, model saved to: " + MODEL_DIR)

## 4.4 Visualisierung Fine-Tuning (pre-trained)

In [ ]:
# summarize history for accuracy
plt.plot(*smooth(history.acc, 100))
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('step')
plt.legend(['train'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(*smooth(history.loss, 100))
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('step')
plt.legend(['train'], loc='upper left')
plt.show()

Führen Sie nun die finale Einzelinferenz auf den Trainings-/Validierungsdaten selbst durch

# Weight Averaging

[Polyak averaging](https://www.researchgate.net/publication/236736759_New_stochastic_approximation_type_procedures) kann auf neuronale Netze angewendet werden, indem ein Durchschnitt der Gewichte verschiedener Modelle gebildet wird. Dies sorgt oft für stabiliere und teilweise sogar bessere Ergebnisse.

Code angepasst von: https://machinelearningmastery.com/polyak-neural-network-model-weight-ensemble/

In [ ]:
from stat import S_ISREG, ST_MODE, ST_MTIME

# Clear session memory to avoid OOM
K.clear_session()

def get_model_filenames(directory, n_max, prefix):
    print("Loading a maximum of {} models with filename prefix {} from directory {}"
          .format(n_max, prefix, directory))
    
    # get all entries in the directory w/ stats
    entries = (os.path.join(directory, fn) for fn in os.listdir(directory))
    entries = ((os.stat(path), path) for path in entries)

    # leave only regular files, insert modification date
    entries = ((stat[ST_MTIME], path)
               for stat, path in entries if S_ISREG(stat[ST_MODE]))

    results = []
    i = 0
    for cdate, path in sorted(entries, reverse = True):
        results.append(os.path.join(directory, path))
        i += 1
        if i >= n_max:
            break
    
    #results.reverse()
    
    print("Found {} models: {}".format(len(results), results))
    
    return results

# load models from file
def load_models_from_directory(directory, n_max, prefix):
    # gather model names from directory
    all_models = list()
    for filename in get_model_filenames(directory, n_max, prefix):
        print('Loading %s...' % filename)
        # load model from file
        model = models.load_model(filename)
        # add to list of members
        all_models.append(model)
        print('Done loading %s' % filename)
    return all_models


# create a model from the weights of multiple models
def model_weight_ensemble(members, weights):
    print("Creating averaged model, this can take a while...")
    # determine how many layers need to be averaged
    n_layers = len(members[0].get_weights())
    # create an set of average model weights
    avg_model_weights = list()
    for layer in range(n_layers):
        print("Processing layer {}".format(layer))
        # collect this layer from each model
        layer_weights = np.array([model.get_weights()[layer] for model in members])
        # weighted average of weights for this layer
        avg_layer_weights = np.average(layer_weights, axis=0, weights=weights)
        # store average layer weights
        avg_model_weights.append(avg_layer_weights)
    # create a new model with the same structure
    model = models.clone_model(members[0])
    # set the weights in the new
    model.set_weights(avg_model_weights)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    print("Finished creating averaged model")
    return model

PREFIX = "pre_trained"
NUM_MODELS = 4
# weight decay factor
ALPHA = 2.0

members = load_models_from_directory(MODEL_DIR, NUM_MODELS, PREFIX)
#members = [models.load_model('/workspace/models/classification/pre_trained_full_12-0.14.hdf5')]


# uncomment for exponential weight decay
#weights = [exp(-i/ALPHA) for i in range(1, len(members)+1)]
# uncommend for linear decay
#weights = [i/n_members for i in range(len(members), 0, -1)]
# uncomment for no weight decay
weights = [1/len(members) for i in range(1, len(members)+1)]
model = model_weight_ensemble(members, weights)

print("Starting evaluation:")
#evaluation = model.evaluate_generator(train_generator, NUM_TRAINING_SAMPLES // BATCH_SIZE)
#print("Training loss:", evaluation[0], "\nTraining accuracy: {:.2f}%".format(evaluation[1]*100))
evaluation = model.evaluate_generator(validation_generator, NUM_VALIDATION_SAMPLES // BATCH_SIZE)
print("Validation loss:", evaluation[0], "\nValidation accuracy: {:.2f}%".format(evaluation[1]*100))

## Out of Class Bilder

Zum Test wie unser Modell mit Bildern umgeht die es noch nie gesehen hat, geben wir ihm nun das folgende Schädel-CT-Bild:

<img src="https://prod-images.static.radiopaedia.org/images/17058746/a60ee77e6b44da2d72a25cea08dfd2_big_gallery.jpeg" />
Quelle: radiopaedia.org

In [ ]:
example_generator = ImageDataGenerator().flow_from_directory(DATA_DIR + '/ooc', batch_size=1, target_size=(128,128), class_mode='categorical')

for x_batch, y_batch in example_generator:
    x_input = x_batch[0]/255.
    x_input = x_input[np.newaxis, ...]
    result = model.predict(x_input)
    
    predicted_class = CLASSES[np.argmax(result[0], axis=-1)]
    correct_class = CLASSES[np.where(y_batch[0] == 1.)[0][0]]
    
    print("\n\nImage", i+1)
    plt.figure()
    plt.imshow(x_batch[0].astype(int))
    
    
    print_prediction_result(result)
    plt.show()
    
    break

# Results

## Run 1

Config:

```python
# Data augmentation
# see https://keras.io/preprocessing/image/
# see https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html
train_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        #interpolation_order=2
        fill_mode='nearest')

# only rescale validation data
validation_datagen = ImageDataGenerator(rescale=1. / 255)

# this is a generator that will read pictures found in subfolers of '/train',
# in the dataset directory and indefinitely generate batches of augmented image
# data
print("Training data:")
train_generator_raw = train_datagen.flow_from_directory(
        DATASET_DIR + '/train',  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 128x128
        batch_size=BATCH_SIZE,
        #interpolation="lanczos",
        class_mode='categorical')
# […]
LEARN_RATE = 0.001
# one epoch training of the heads:
optimizer = tf.keras.optimizers.Adam(lr=LEARN_RATE, decay=0.0025, clipnorm=1.)
# 100 epochs training full network
optimizer = tf.keras.optimizers.SGD(lr=LEARN_RATE, decay=0.00025, clipnorm=1.)
```

Weight averaging mit 5 Modellen:

Validation loss: ~0.26  
Validation accuracy: ~94%

## Run 2

Gleiche Konfiguration wie Run 1 außer:

- ```interpolation="lanczos"```

Weight averaging mit 5 Modellen:

Validation loss: 0.2383039555444801  
Validation accuracy: 95.26%

## Run 3

Gleiche Konfiguration wie Run 2 außer:

- `Adam` Optimizer für Fine-Tuning anstelle von `SGD`

Weight averaging mit 5 Modellen:

Validation loss: 0.183909532866773  
Validation accuracy: 94.50%

## Run 4

Gleiche Konfiguration wie Run 2 außer:

- Bildgröße auf 400x400 erhöht
- Batch-Size halbiert (8)
- 80 Epochen training
- `clipnorm=1.` beim Optimizer nach 40 Epochen entfernt

Weight averaging mit 5 Modellen (nach 80 Epochen Training):

Validation loss: 0.15276804335761035  
Validation accuracy: 96.99%

Single best model:

Validation loss: 0.1423978957033721  
Validation accuracy: 97.06%


# TODO

- Kreuzvalidierung


In [ ]:
print(MODEL_DIR)

In [ ]:
ls /workspace/models/classification